In [56]:
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.pauli import group_observables

from scipy.optimize import minimize

import os, json, time, logging
import numpy as np
from datetime import datetime, timedelta

from multiprocessing import Pool

from susy_qm import calculate_Hamiltonian, ansatze

In [97]:
potential="AHO"
cutoff=64
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues, eigenvectors = np.linalg.eig(H)

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = eigenvectors[:, min_index]

min_eigenvalue

np.complex128(-1.9425227038037043e-08+0j)

In [58]:
path = r"C:\Users\johnkerf\Documents\Quantum-Computing\SUSY\SUSY QM\PennyLane\COBYQA\qml.expval\Adaptive-VQE\StatevectorFiles\AHO\data_32.txt"

with open(path, "r", encoding="utf-8") as f:
            text = f.read()

json_part = text[: text.rfind("}") + 1]
data = json.loads(json_part)

In [59]:
params = []
for gate in data["circuit"]:
    p = gate["param"]
    params.append(p)

params

[1.3481621295000075,
 -0.21653160259712087,
 -0.4369331245775947,
 -0.31646953135696987,
 -0.11983294485674534,
 0.30680681956842537,
 0.06735205302740202,
 -0.004356383247931526,
 -0.17128465698238843,
 -0.061558396733602125,
 -1.3226111723119027,
 0.5644682753211268,
 0.12759589336155022,
 0.7167899149644672,
 -0.2561571256913027,
 0.3037971747490715,
 0.003952491894855683]

In [107]:
params = [
            0.43082852479269595,
            4.609663795130995,
            5.424494214705023,
            1.1406478450976434,
            2.980875002581941,
            2.315432156175172,
            0.40020394062345765,
            2.334810797368153,
            1.1860253778460401,
            5.084280455691308,
            0.5251677404620336,
            4.289355017781702,
            6.283185307179586,
            5.401868203181394,
            0.5952620834951343,
            0.761115156197762,
            5.798468440261053,
            4.67511449049363,
            0.6660142089677192,
            0.6352085553346316,
            5.2656863038981925,
            4.458578247588953,
            0.9445287849051023,
            3.5132718034687667,
            3.5571202655985212,
            2.7474320993347403
        ]

In [108]:
ansatz = ansatze.get("CQAVQE_AHO64_exact")

num_qubits = 7
num_params = ansatz.n_params
max_gate = len(params)+1

pms = params[:max_gate]

gates = ansatze.truncate_ansatz(ansatz, pms, num_qubits, max_gate=max_gate)

dev = qml.device("default.qubit", wires=num_qubits, shots=100000)
@qml.qnode(dev, cache=False)
def circuit():
    #ansatz(params, num_qubits)
    for gate in gates:
            qml.apply(gate)

    return qml.counts(wires=list(range(num_qubits)))
    #return qml.state()
    #return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

samples = circuit()
samples

{np.str_('1000000'): np.int64(19523),
 np.str_('1000010'): np.int64(3749),
 np.str_('1000100'): np.int64(9062),
 np.str_('1000110'): np.int64(1212),
 np.str_('1001000'): np.int64(42521),
 np.str_('1001010'): np.int64(1564),
 np.str_('1001100'): np.int64(2221),
 np.str_('1001110'): np.int64(2284),
 np.str_('1010000'): np.int64(7763),
 np.str_('1010010'): np.int64(158),
 np.str_('1010100'): np.int64(99),
 np.str_('1010110'): np.int64(1403),
 np.str_('1011000'): np.int64(59),
 np.str_('1011010'): np.int64(6846),
 np.str_('1011100'): np.int64(1430),
 np.str_('1011110'): np.int64(106)}

In [109]:
sorted_states = sorted(samples.items(), key=lambda x: x[1], reverse=True)
print(sorted_states)

top_states = [s for s, c in sorted_states]
#print("Selected:", top_states)

idx = [int(s, 2) for s in top_states]
#print("Indices:", idx)

H_reduced = H[np.ix_(idx, idx)]
#print("Reduced Hamiltonian:\n", H_reduced)


[(np.str_('1001000'), np.int64(42521)), (np.str_('1000000'), np.int64(19523)), (np.str_('1000100'), np.int64(9062)), (np.str_('1010000'), np.int64(7763)), (np.str_('1011010'), np.int64(6846)), (np.str_('1000010'), np.int64(3749)), (np.str_('1001110'), np.int64(2284)), (np.str_('1001100'), np.int64(2221)), (np.str_('1001010'), np.int64(1564)), (np.str_('1011100'), np.int64(1430)), (np.str_('1010110'), np.int64(1403)), (np.str_('1000110'), np.int64(1212)), (np.str_('1010010'), np.int64(158)), (np.str_('1011110'), np.int64(106)), (np.str_('1010100'), np.int64(99)), (np.str_('1011000'), np.int64(59))]


In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(H_reduced)

mi = np.argmin(eigenvalues)
me = eigenvalues[mi]
mv = eigenvectors[:, mi]

me

In [111]:
me

np.complex128(1.833514049596847e-05+0j)

In [19]:
min_eigenvalue

np.complex128(0.8915993623272539+0j)